# **Random Forest**
- Using data that was not annotated
- Training it on the unbalanced data
- Using avgpool
- 6 second window

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Standard libraries
import numpy as np
import pandas as pd
import time
import os

# For audio
from IPython.display import Audio
import librosa

# For preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

# For modeling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Operational
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import scipy.ndimage
import pygame
import time
from scipy.signal import butter, filtfilt
import random

pygame 2.6.1 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
pkl_path = '/content/drive/My Drive/Final-Year-Project/Dataset/Project-V4/feature-extraction/NotAnnotated/NotAveragePooled/split_features_6s_all.pkl'

# Load the pickle file
with open(pkl_path, 'rb') as file:
    data = pickle.load(file)

In [4]:
train_data = data['train'].copy()
val_data = data['val'].copy()

In [5]:
train_data

{'melspectrogram': array([[[2.70414866e-03, 2.79165306e-04, 2.78151976e-06, ...,
          2.09090982e-14, 1.00035500e-07, 4.10201886e-06],
         [2.85087147e-03, 2.95306839e-04, 3.42857310e-06, ...,
          2.42591675e-14, 1.00793242e-07, 4.13385302e-06],
         [3.04498922e-03, 3.15533762e-04, 4.71718391e-06, ...,
          3.06456201e-14, 1.02097112e-07, 4.18868937e-06],
         ...,
         [4.71950382e-06, 6.96979924e-07, 9.83989841e-06, ...,
          4.76671411e-08, 2.59462464e-08, 1.32848568e-07],
         [4.43196142e-06, 4.70733005e-07, 1.06044568e-07, ...,
          2.24950763e-10, 2.39690650e-09, 9.34205219e-08],
         [4.34125244e-06, 4.54310514e-07, 3.83112779e-09, ...,
          6.16144049e-13, 1.95562647e-09, 8.06973052e-08]],
 
        [[1.73426295e-05, 3.63408438e-06, 4.80290598e-13, ...,
          1.60876280e-14, 7.00188949e-08, 1.07775013e-06],
         [1.84753151e-05, 3.87795431e-06, 5.39064175e-13, ...,
          2.48021127e-14, 7.05198033e-08, 1.0866

In [6]:
train_labels = train_data['label'].copy()
temp = train_data.copy()
del temp['label']
tr_features = temp

In [7]:
val_labels = val_data['label'].copy()
temp = val_data.copy()
del temp['label']
v_features = temp

# Shuffle Data

In [8]:
def shuffle_data(input_label, input_features):
  input_len = len(input_label)
  np.random.seed(1826)
  input_indices = np.random.permutation(input_len)
  input_features = {key: np.array([input_features[key][i] for i in input_indices]) for key in input_features} # dictionary comprehension
  input_label = np.array([input_label[i] for i in input_indices])

  return input_label, input_features

In [9]:
train_y, train_features = shuffle_data(train_labels, tr_features)

In [10]:
val_y, val_features = shuffle_data(val_labels, v_features)

In [11]:
display(train_y.shape)
display(train_y[:15])

(7105,)

array([ 1, 12, 12, 18,  0,  6,  9,  2, 15,  9, 15,  0,  9,  2, 12],
      dtype=int32)

In [12]:
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)
  display(train_features[key][0])

'melspectrogram'

(7105, 60, 259)

array([[2.69040937e-03, 4.08457172e-04, 3.47520585e-13, ...,
        1.70315054e-13, 7.40936271e-06, 1.18791981e-04],
       [2.70075129e-03, 4.10013696e-04, 3.93403282e-13, ...,
        1.76836050e-13, 7.51821703e-06, 1.20539639e-04],
       [2.71822099e-03, 4.12641652e-04, 4.86951418e-13, ...,
        1.91108949e-13, 7.70506226e-06, 1.23548468e-04],
       ...,
       [2.08812007e-04, 3.18310420e-05, 4.35271934e-08, ...,
        4.00302065e-08, 1.74103692e-06, 2.63066062e-05],
       [1.88873108e-04, 2.87703725e-05, 1.76288301e-10, ...,
        1.57014632e-10, 1.42424630e-06, 2.29200435e-05],
       [1.77336857e-04, 2.70143104e-05, 2.32370972e-13, ...,
        1.17086386e-13, 1.32333290e-06, 2.13012012e-05]])

'mfcc'

(7105, 20, 259)

...

(7105, 12, 259)

array([[0.24864305, 0.29631535, 0.31045966, ..., 0.40069001, 0.50801101,
        0.38144471],
       [0.24534524, 0.28840458, 0.33379185, ..., 0.42041354, 0.50586702,
        0.62016281],
       [0.15298406, 0.11485447, 0.08231069, ..., 0.53125893, 0.64276512,
        0.46286747],
       ...,
       [1.        , 0.63096119, 0.77352367, ..., 0.64788047, 0.5271435 ,
        0.41385586],
       [0.97426447, 1.        , 1.        , ..., 0.45275858, 0.49202924,
        0.42027346],
       [0.27945555, 0.30617621, 0.26337507, ..., 0.58270217, 1.        ,
        0.5825718 ]])

'rms'

(7105, 1, 259)

array([[0.01612907, 0.02036163, 0.02236381, 0.02032115, 0.01881817,
        0.01514378, 0.01315711, 0.01117222, 0.01068601, 0.01038471,
        0.01493485, 0.02178646, 0.02435838, 0.02619782, 0.02583299,
        0.02277914, 0.02014609, 0.01751793, 0.01387478, 0.00852018,
        0.00690586, 0.00614171, 0.00770546, 0.01995697, 0.03698479,
        0.0426441 , 0.04647807, 0.04581114, 0.03440014, 0.02740226,
        0.01983235, 0.01073553, 0.0076463 , 0.00661767, 0.00606081,
        0.00634777, 0.00919547, 0.01277758, 0.01689507, 0.02074126,
        0.0227426 , 0.02257645, 0.02067827, 0.01722183, 0.01285113,
        0.00976115, 0.00711595, 0.00767853, 0.00995252, 0.01214419,
        0.01481564, 0.01701612, 0.01691237, 0.01644171, 0.01501546,
        0.01161532, 0.01009744, 0.00841241, 0.00682253, 0.00637262,
        0.00604512, 0.00532832, 0.00585929, 0.01020321, 0.01224944,
        0.01383804, 0.01583311, 0.01574408, 0.01638243, 0.01634659,
        0.01471977, 0.01260509, 0.00995492, 0.00

In [13]:
display(val_y.shape)
display(val_y[:15])

(1896,)

array([15,  2,  2,  2,  0, 18,  2,  7, 17,  3,  2, 19,  2, 19,  0],
      dtype=int32)

In [14]:
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)
  display(val_features[key][0])

'melspectrogram'

(1896, 60, 259)

array([[1.93338063e-03, 1.98195157e-04, 1.73273927e-13, ...,
        2.43286351e-17, 1.53878134e-09, 4.13960773e-06],
       [1.95305293e-03, 2.00203552e-04, 3.07182569e-13, ...,
        4.05249738e-17, 1.54097439e-09, 4.14508204e-06],
       [1.98668457e-03, 2.03637224e-04, 8.56416508e-13, ...,
        1.40852237e-16, 1.54199312e-09, 4.15323324e-06],
       ...,
       [2.94063560e-05, 3.43539450e-06, 4.52336753e-07, ...,
        3.50377592e-10, 1.49650976e-10, 1.99068144e-08],
       [1.87940410e-05, 1.92592488e-06, 1.87399595e-09, ...,
        1.62022206e-12, 2.91367028e-12, 4.34889919e-09],
       [1.43345861e-05, 1.46661767e-06, 1.48607980e-12, ...,
        8.18254333e-15, 3.89491302e-13, 1.03254425e-09]])

'mfcc'

(1896, 20, 259)

...

(1896, 12, 259)

array([[1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        1.04429035e-02, 2.32952201e-04, 1.81646110e-02],
       [3.90740244e-01, 3.20093565e-01, 5.87131300e-01, ...,
        4.22160377e-02, 1.99507003e-03, 2.90285305e-02],
       [3.92294799e-01, 2.52268114e-01, 6.10323358e-01, ...,
        4.90544230e-01, 1.83283318e-01, 1.27773882e-01],
       ...,
       [3.36610441e-01, 1.41244550e-01, 2.44668984e-01, ...,
        2.91197295e-02, 3.15024306e-03, 2.36044398e-02],
       [2.41128538e-01, 1.86966151e-01, 2.51816555e-01, ...,
        3.96117972e-03, 1.00874285e-03, 2.12197038e-02],
       [4.52456962e-01, 4.53742162e-01, 4.17860162e-01, ...,
        2.42732364e-03, 3.09255736e-04, 1.59454829e-02]])

'rms'

(1896, 1, 259)

array([[0.00062057, 0.00081435, 0.00100355, 0.00101341, 0.0009582 ,
        0.0008615 , 0.0006986 , 0.00064286, 0.00066621, 0.00070287,
        0.00073753, 0.00074409, 0.00074991, 0.00075677, 0.00079612,
        0.02027168, 0.04253102, 0.04830509, 0.04830541, 0.04385286,
        0.02291447, 0.00078997, 0.00082059, 0.00097988, 0.00100605,
        0.00095852, 0.00085913, 0.00066692, 0.00063826, 0.00064309,
        0.0006566 , 0.01604473, 0.03785714, 0.0458034 , 0.04594636,
        0.04622704, 0.03680744, 0.02719475, 0.02695601, 0.02107304,
        0.00711193, 0.00102109, 0.00092804, 0.00080745, 0.00078223,
        0.00076144, 0.00072847, 0.00064887, 0.00061626, 0.00061098,
        0.00062443, 0.00063061, 0.00062421, 0.00063657, 0.00068225,
        0.0006858 , 0.00067399, 0.0006922 , 0.00067687, 0.00642227,
        0.01179406, 0.01477852, 0.0150725 , 0.01366344, 0.00945538,
        0.01229848, 0.02199678, 0.0236273 , 0.02367615, 0.04526687,
        0.06944718, 0.07077155, 0.07075211, 0.05

## **Random Forest Model**

In [15]:
train_results = {}
val_results = {}

### **With all the features**

In [16]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['chroma'], train_features['rms'], train_features['melspectrogram']), axis=1)
# A random forest model expect a 2D input of (n_samples, n_features). Since the input is 3D here, it will cause an error when passed through a RF,
# I need to flatten the training features from 3D to 2D... eg. (7105, 93 * 259)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)

training_features.shape

(7105, 24087)

In [17]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['chroma'], val_features['rms'], val_features['melspectrogram']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(1896, 24087)

Fit the model with training data

In [18]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [19]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['all_features'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['all_features'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.36761603375527424


In [20]:
train_yhat_result = rf.predict(training_features)

print('Training classification Report \n')
print(classification_report(train_y, train_yhat_result))

Training classification Report 

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1017
           1       1.00      1.00      1.00       727
           2       1.00      1.00      1.00       501
           3       1.00      1.00      1.00       462
           4       1.00      1.00      1.00        66
           5       1.00      1.00      1.00       116
           6       1.00      1.00      1.00       123
           7       1.00      1.00      1.00       403
           8       1.00      1.00      1.00       108
           9       1.00      1.00      1.00       197
          10       1.00      1.00      1.00       282
          11       1.00      1.00      1.00       107
          12       1.00      1.00      1.00       718
          13       1.00      1.00      1.00       183
          14       1.00      1.00      1.00       265
          15       1.00      1.00      1.00       403
          16       1.00      1.00      1.00     

In [21]:
val_yhat_result = rf.predict(validation_features)

print('Validation classification Report \n')
print(classification_report(val_y, val_yhat_result))

Validation classification Report 

              precision    recall  f1-score   support

           0       0.32      0.72      0.44       272
           1       0.24      0.47      0.32       118
           2       0.87      0.30      0.44       476
           3       0.52      0.84      0.64        92
           4       0.17      0.02      0.04        49
           5       0.23      0.15      0.18        20
           6       0.00      0.00      0.00        18
           7       0.45      0.51      0.48       111
           8       0.00      0.00      0.00        37
           9       0.82      0.65      0.72        65
          10       0.28      0.24      0.26        75
          11       0.00      0.00      0.00        37
          12       0.28      0.57      0.38        89
          13       0.25      0.03      0.06        31
          14       0.02      0.05      0.03        19
          15       0.22      0.10      0.13       136
          16       0.27      0.21      0.24   

### **With MFCCs only**

In [22]:
training_features_3D = train_features['mfcc']
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(7105, 5180)

In [23]:
validation_features_3D = val_features['mfcc']
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(1896, 5180)

In [24]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [25]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.34810126582278483


### **With Mel-Spectrogram Only**

In [26]:
training_features_3D = train_features['melspectrogram']
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(7105, 15540)

In [27]:
validation_features_3D = val_features['melspectrogram']
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(1896, 15540)

In [28]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [29]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.30748945147679324


## **With Chroma Only**

In [30]:
training_features_3D = train_features['chroma']
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(7105, 3108)

In [31]:
validation_features_3D = val_features['chroma']
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(1896, 3108)

In [32]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [33]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['chroma'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['chroma'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.21255274261603377


## **With MFCCs and Mel-Spectrogram**

In [34]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['melspectrogram']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(7105, 20720)

In [35]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['melspectrogram']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(1896, 20720)

In [36]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [37]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc_melspectrogram'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc_melspectrogram'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.36075949367088606


## **With MFCCs and Chroma**

In [38]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['chroma']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(7105, 8288)

In [39]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['chroma']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(1896, 8288)

In [40]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [41]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc_chroma'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc_chroma'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.33175105485232065


## **With MFCCs and RMS**

In [42]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['rms']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(7105, 5439)

In [43]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['rms']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(1896, 5439)

In [44]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [45]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc_rms'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc_rms'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3560126582278481


## **With Chroma and RMS**

In [46]:
training_features_3D = np.concatenate((train_features['chroma'], train_features['rms']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(7105, 3367)

In [47]:
validation_features_3D = np.concatenate((val_features['chroma'], val_features['rms']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(1896, 3367)

In [48]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [49]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['chroma_rms'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['chroma_rms'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.26371308016877637


## **With Mel-Spectrogram and Chroma**

In [50]:
training_features_3D = np.concatenate((train_features['melspectrogram'], train_features['chroma']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(7105, 18648)

In [51]:
validation_features_3D = np.concatenate((val_features['melspectrogram'], val_features['chroma']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(1896, 18648)

In [52]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [53]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_chroma'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_chroma'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3154008438818565


## **With Mel-Spectrogram and RMS**

In [54]:
training_features_3D = np.concatenate((train_features['melspectrogram'], train_features['rms']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(7105, 15799)

In [55]:
validation_features_3D = np.concatenate((val_features['melspectrogram'], val_features['rms']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(1896, 15799)

In [56]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [57]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_rms'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_rms'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.34968354430379744


## **With Mel-Spectrogram, MFCCs, RMS**

In [58]:
training_features_3D = np.concatenate((train_features['melspectrogram'], train_features['rms'], train_features['mfcc']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(7105, 20979)

In [59]:
validation_features_3D = np.concatenate((val_features['melspectrogram'], val_features['rms'], val_features['mfcc']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(1896, 20979)

In [60]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [61]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_rms_mfcc'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_rms_mfcc'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3691983122362869


## **With Mel-Spectrogram, MFCCs, Chroma**

In [62]:
training_features_3D = np.concatenate((train_features['melspectrogram'], train_features['chroma'], train_features['mfcc']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(7105, 23828)

In [63]:
validation_features_3D = np.concatenate((val_features['melspectrogram'], val_features['chroma'], val_features['mfcc']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(1896, 23828)

In [64]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [65]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_chroma_mfcc'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_chroma_mfcc'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3686708860759494


# Review results from all models

In [66]:
train_results_df = pd.DataFrame(list(train_results.items()), columns=['Features', 'Train_Accuracy']).round(2)
val_results_df = pd.DataFrame(list(val_results.items()), columns=['Features', 'Val_Accuracy']).round(2)

result_df = train_results_df.merge(val_results_df, on='Features')
result_df = result_df.sort_values('Features')
result_df

,Features,Train_Accuracy,Val_Accuracy
0,all_features,1.0,0.37
3,chroma,1.0,0.21
7,chroma_rms,1.0,0.26
2,melspectrogram,1.0,0.31
8,melspectrogram_chroma,1.0,0.32
11,melspectrogram_chroma_mfcc,1.0,0.37
9,melspectrogram_rms,1.0,0.35
10,melspectrogram_rms_mfcc,1.0,0.37
1,mfcc,1.0,0.35
5,mfcc_chroma,1.0,0.33
